In [ ]:
from http import client
from urllib import response
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from img2vec_pytorch import Img2Vec
from sentence_transformers import SentenceTransformer
import plotly.express as px
import plotly.graph_objects as go
import urllib.request
import requests
import pandas as pd
import json
import pprint
import seaborn as sns
import matplotlib.pyplot as plt
import json
import numpy as np

In [4]:
desc_emb = np.load('./data/desc_emb.npz')
desc_embeddings = dict(zip(("desc_embeddings"), (desc_emb[k] for k in desc_emb)))['d']
print(desc_embeddings.shape)

(11083, 768)


In [26]:
#Test
desc_embeddings = desc_embeddings[:1000]
description_list = description_list[:1000]
contract_addresses = contract_addresses[:1000]
name_list = name_list[:1000]

In [ ]:
nft_tsne = TSNE(n_components=3, n_jobs = -1).fit_transform(desc_embeddings)
nft_tsne_df = pd.DataFrame(nft_tsne)
nft_tsne_df = (nft_tsne_df-nft_tsne_df.mean())/nft_tsne_df.std()
nft_tsne_df

In [ ]:
truncated_descriptions = [desc[0:50] + '...' if len(desc) > 50 else desc for desc in description_list]
final_df = pd.concat([nft_tsne_df, pd.DataFrame(contract_addresses), pd.DataFrame(name_list), pd.DataFrame(truncated_descriptions)], axis = 1)
final_df.columns = ['X', 'Y', 'Z', 'NFT_Address', 'Name', 'D']
final_df

In [ ]:
fig = px.scatter_3d(
    final_df, x= 'X', y= 'Y', z= 'Z',
    size=[1]*len(final_df),
    color = 'NFT_Address', 
    labels= {'influence_rating': 'Legend'},
    hover_data = 'D',
)
fig.show()

In [ ]:
fig.update_layout(xaxis=dict(showspikes=False) , 
        width=2000,
        height=1000,
        margin=dict(
            l=2,
            r=2,
            b=2,
            t=2,
            pad=4
        ), )

fig.update_traces(marker_size = 3 )
fig.update_coloraxes(showscale=False)
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )

fig.write_html("attributes+description_pca.html")
fig.show()

In [18]:
# Testing Random Neighborhood
index = np.random.randint(0, len(desc_embeddings))
sample = desc_embeddings[index]
min_index = 0
min_distance = 10

for i, di in enumerate(desc_embeddings):
    distance = np.sum(np.square((sample - di)))
    if distance < min_distance and index != i:
        min_distance = distance
        min_index = i

print(f'The closest NFT is {min_index} with a distance of {min_distance}')
print(f'\ndescription {min_index}: {description_list[min_index]}')
print(f'\ndescription {index}: {description_list[index]}')

The closest NFT is 688 with a distance of 0.27132129669189453

description 688: alien enemes are a collection of 10,000 randomly generated aliens provides you not only a mind-blowing PFP and also membership in our exclusive club for meets, events, exclusive merch, 2D Comic Books and more roadmap versions will be released as we progress!

description 6725: SCAR is a collection of 10,200 utility-enabled NFTs. Hold Aliens and Enjoy a variety of benefits!
